In [13]:
import pandas as pd
import sys
import requests
from io import StringIO
from fuzzywuzzy import fuzz
from tqdm.auto import tqdm
import re

sys.path.append("../src")
from data_fetching.sirene import get_company_info

# Idée générale

Il s'agit dans ce notebook de mettre en place l'idée de cet [issue](https://github.com/dataforgoodfr/offseason_missiontransition_simulateur/issues/8). 

Formellement, entraîner un modèle qui à un couple `(aid, company)` est en mesure de rendre un `score` entre 0 et 1 representant les chances de succès de la demande d'aide pour savoir à quel point il est pertinent de proposer précisément cette `aid` à cette `company`.

# Préparation du dataset

## Données ADEME
Nous disposons d'un ensemble d'exemples de $n$ couples $\mathcal{S} = \{(a_i, c_i)\} _{i\leq n}$ qui ont fait l'objet d'une attribution de l'ADEME : pour tout $i\leq n$, l'entreprise $c_i$ a reçu l'aide $a_i$ proposée par l'ADEME.

Nous voulons créer un dataset labelisé, $\mathcal{D} = \{[(a_i, c_i), y_i]\}_{i\leq N}$ où $y_i$ représente les chances que l'entreprise $c_i$ obtienne (ou ait intérêt à obtenir) l'aide $a_i$.

$\mathcal{S}$ nous fournit un premier dataset avec des labels $y_i=1$, puisqu'il s'agit d'exemples où l'entreprise a effectivement obtenu l'aide en question.

Pour avoir un dataset complet, il faudrait avoir des exemples de couples $(a_i, c_i)$, avec un score $y_i$ inférieur à 1. 

## Exploration du dataset

In [14]:
### Dataset preparation

ademe_mt = pd.read_csv("../first_ademe_mt.csv")

In [15]:
ademe_mt.columns

Index(['index_ademe', 'Nom de l attribuant_ademe', 'idAttribuant_ademe',
       'dateConvention_ademe', 'referenceDecision_ademe',
       'nomBeneficiaire_ademe', 'idBeneficiaire_ademe', 'objet_ademe',
       'montant_ademe', 'nature_ademe', 'conditionsVersement_ademe',
       'datesPeriodeVersement_ademe', 'idRAE_ademe', 'notificationUE_ademe',
       'pourcentageSubvention_ademe', 'location.lat_ademe',
       'location.lon_ademe', 'DEPET_ademe', 'naf1etlib_ademe',
       'naf2etlib_ademe', 'naf3etlib_ademe', 'naf4etlib_ademe',
       'naf5etlib_ademe', 'id_mt', 'name_mt', 'perimeter_mt', 'goal_mt',
       'beneficiary_mt', 'aidDetails_mt', 'eligibility_mt', 'conditions_mt',
       'fundingSourceUrl_mt', 'applicationEndDate_mt', 'applicationUrl_mt',
       'slug_mt', 'environmentalTopics_mt', 'fundingTypes_mt', 'regions_mt',
       'contactGuidelines_mt', 'subventionRateUpperBound_mt',
       'subventionRateLowerBound_mt', 'loanAmount_mt',
       'applicationStartDate_mt', 'projectExa

In [16]:
df = ademe_mt[["idBeneficiaire_ademe", "nomBeneficiaire_ademe", "naf5etlib_ademe", "id_mt", "name_mt", "aidDetails_mt", "eligibility_mt"]]

Se pose maintenant la question des variables à utiliser pour le modèle, côté aide on a le nom, la description et l'éligibilité. Côté Entreprise on a le secteur d'activité, et les infos récupérables depuis l'API SIRENE.

## Augmentation du dataset

L'idée est donc la suivante : pour chaque aide $a_i$ dans un couple dans $\mathcal{S}$, on sait que seule l'entreprise $c_i$ a obtenu cette aide, autrement dit tous les couples $(a_i, c_j)$ pour $j\neq i$ peuvent être vu comme des exemples avec un label $y=0$.

Ceci nous fournit donc $n(n-1)$ exemples avec un label $y=0$ ($n-1$ exemples pour les $n$ aides). Pour éviter d'avoir un dataset trop déséquilibré, on peut en garder $n$ au hasard afin de former un dataset de taille $2n$.

In [17]:
print(len(df))
print(len(df.id_mt.unique()))

149
14


In [18]:
def augment(df, company_columns, aid_columns):
    new_df = pd.DataFrame()
    df_ = df.copy()
    df_["label"] = 1
    for id in df_["idBeneficiaire_ademe"].unique():
        global rows
        rows = df_.groupby("idBeneficiaire_ademe").get_group(id)
        new_rows = pd.DataFrame()
        for aid_id in df_["id_mt"].unique():
            if aid_id not in rows["id_mt"]:
                global aid
                aid = df_.groupby("id_mt").get_group(aid_id)
                new_row = pd.DataFrame({
                    **{name:[value] for name, value in rows[company_columns].iloc[0].items()},
                    **{name:[value] for name, value in aid[aid_columns].iloc[0].items()},
                    "label": [0]
                })
                new_rows = new_rows.append(new_row, ignore_index=True)
        new_rows = rows.append(new_rows.sample(1), ignore_index=True)
        new_df = new_df.append(new_rows, ignore_index=True)
    return new_df



In [19]:
company_columns = [
    "idBeneficiaire_ademe",
    "nomBeneficiaire_ademe",
    "naf5etlib_ademe"
]
aid_columns = [
    "id_mt",
    "name_mt",
    "aidDetails_mt",
    "eligibility_mt"
]
augmented_df = augment(df, company_columns, aid_columns)

In [20]:
augmented_df.head()

,idBeneficiaire_ademe,nomBeneficiaire_ademe,naf5etlib_ademe,id_mt,name_mt,aidDetails_mt,eligibility_mt,label
0,7.269205e+13,CLEMENT SAS,Fabrication de machines-outils pour le travail...,44,Financer vos projets de transition écologique ...,<p>\n <em>\n Financer vos projets de transiti...,<p>\n L'aide est destinée à toutes les TPE et ...,1
1,7.269205e+13,CLEMENT SAS,Fabrication de machines-outils pour le travail...,163,Financer les installations de récupération de ...,<p>\n <strong>\n Aide - « Financement d'insta...,<p>\n Ne sont pas éligibles à cette aide : les...,0
2,3.950618e+13,INDUSTRIE DOLOISE DE MICRO-MECANIQUE,Mécanique industrielle,44,Financer vos projets de transition écologique ...,<p>\n <em>\n Financer vos projets de transiti...,<p>\n L'aide est destinée à toutes les TPE et ...,1
3,3.950618e+13,INDUSTRIE DOLOISE DE MICRO-MECANIQUE,Mécanique industrielle,274,Développer les énergies renouvelables thermiques,<p>\n <strong>\n Aide - « Contrat de développ...,<p>\n <strong>\n Critères d'éligibilité :\n <...,0
4,3.391339e+13,TRANSDEV BFC SUD,Transports routiers réguliers de voyageurs,44,Financer vos projets de transition écologique ...,<p>\n <em>\n Financer vos projets de transiti...,<p>\n L'aide est destinée à toutes les TPE et ...,1


# Modélisation

## Modèle

Les features en langage naturel peuvent être traités avec [SBERT](https://www.sbert.net/docs/quickstart.html) pour ensuite entraîner un modèle de classification.

In [29]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

In [28]:
model_name = "camembert-base" # Pre-trained model for french language
tokenizer = AutoTokenizer.from_pretrained("camembert-base") 
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

Downloading: 100%|██████████| 518M/518M [00:20<00:00, 26.5MB/s] 
All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

Some layers of TFCamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
df

,idBeneficiaire_ademe,nomBeneficiaire_ademe,naf5etlib_ademe,id_mt,name_mt,aidDetails_mt,eligibility_mt
0,7.269205e+13,CLEMENT SAS,Fabrication de machines-outils pour le travail...,44,Financer vos projets de transition écologique ...,<p>\n <em>\n Financer vos projets de transiti...,<p>\n L'aide est destinée à toutes les TPE et ...
1,3.950618e+13,INDUSTRIE DOLOISE DE MICRO-MECANIQUE,Mécanique industrielle,44,Financer vos projets de transition écologique ...,<p>\n <em>\n Financer vos projets de transiti...,<p>\n L'aide est destinée à toutes les TPE et ...
2,3.391339e+13,TRANSDEV BFC SUD,Transports routiers réguliers de voyageurs,44,Financer vos projets de transition écologique ...,<p>\n <em>\n Financer vos projets de transiti...,<p>\n L'aide est destinée à toutes les TPE et ...
3,4.999836e+13,LOCA TRAVAUX EURL,Location et location-bail de machines et équip...,44,Financer vos projets de transition écologique ...,<p>\n <em>\n Financer vos projets de transiti...,<p>\n L'aide est destinée à toutes les TPE et ...
4,3.008465e+13,RODESCHINI SAS,Travaux de maçonnerie générale et gros œuvre d...,44,Financer vos projets de transition écologique ...,<p>\n <em>\n Financer vos projets de transiti...,<p>\n L'aide est destinée à toutes les TPE et ...
...,...,...,...,...,...,...,...
144,2.159025e+13,COMMUNE DE FOURMIES,Administration publique générale,274,Développer les énergies renouvelables thermiques,<p>\n <strong>\n Aide - « Contrat de développ...,<p>\n <strong>\n Critères d'éligibilité :\n <...
145,5.205843e+13,GAEC SAINT DOMICE,"Culture de céréales (à l'exception du riz), de...",267,Financer une assistance à maîtrise d’ouvrage p...,<p>\n <strong>\n Aide - « Financement d'une a...,<p>\n Critères d'éligibilité :\n</p>\n<p>\n L'...
146,5.244164e+13,SOCIETE DE PRODUCTION D'ELECTRICITE ET DE VAPE...,Production d'électricité,182,Financer des « Études préalables à la construc...,<p>\n Avant d'investir dans une installation d...,<p>\n L'aide s'adresse aux collectivités et à ...
147,8.129823e+13,SARL LES BOUEIGES,Production d'électricité,182,Financer des « Études préalables à la construc...,<p>\n Avant d'investir dans une installation d...,<p>\n L'aide s'adresse aux collectivités et à ...


In [50]:
for feature, value in natural_features:
    print(feature, df[feature].str.split(" ").apply(lambda x:len(x) if x==x else 0).mean())

naf5etlib_ademe 5.060402684563758
name_mt 17.06711409395973
aidDetails_mt 305.3288590604027
eligibility_mt 38.22818791946309


In [51]:
natural_features = [
    ("naf5etlib_ademe", 10),
    ("name_mt", 30),
    ("aidDetails_mt", 512),
    ("eligibility_mt", 50)
]

In [52]:
batches = dict()
for feature, value in natural_features:
    batches[feature] = tokenizer(
        df["naf5etlib_ademe"].tolist(),
        padding=True, truncation=True, max_length=value)